todo: 
- letní filmy, co už nedávají (víno)
- záhada hlavolamu je seriál

In [522]:
import os
import polars as pl

In [523]:
pl.Config(tbl_rows=200)

In [524]:
from calendar import monthrange

In [525]:
def monthday(date_value):
    # Here date_value is a datetime.datetime object, not a Polars column
    month_length = monthrange(date_value.year, date_value.month)[1]
    return date_value.month + (date_value.day / month_length)

In [526]:
df = pl.read_parquet(
    os.path.join("data","fdb.parquet")
).with_columns(
    pl.col("datum").dt.month().alias("mesic")
).with_columns(
    pl.col("datum").dt.week().alias("tyden")
).with_columns(
    pl.col("datum").dt.year().alias("rok")
).with_columns(
    pl.when(
        pl.col("url").str.contains("serial/")
    ).then(
        pl.lit("serial")
    ).when(
        pl.col("url").str.contains("film/")
    ).then(
        pl.lit("film")
    ).alias(
        "kategorie"
    )
).sort(
    by="datum"
)

In [527]:
df.head(2)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT2""",2007-01-01 10:05:00,"""Dotek motýla (?) [TV film]""","""05.10.2020 v 12:30 (ČT 3)""","""Další díl/opakování:05.10.2020…","""film/dotek-motyla/39629""",1,1,2007,"""film"""
"""ČT1""",2007-01-01 18:05:00,"""O velkém nosu (?) [TV inscenac…","""01.06.2024 v 13:05 (ČT1)""","""Další díl/opakování:01.06.2024…","""film/o-velkem-nosu/41025""",1,1,2007,"""film"""


In [528]:
df.tail(2)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Prima Show""",2025-04-17 23:55:00,"""Tyhle šaty beru!""","""19.04.2025 v 06:00 (Prima Show…","""Další díl/opakování:19.04.2025…","""film/tyhle-saty-beru-say-yes-t…",4,16,2025,"""film"""
"""Nova Fun""",2025-04-17 23:55:00,"""Dva a půl chlapa""","""18.04.2025 v 12:15 (Nova Fun)1…","""Další díl/opakování:18.04.2025…","""serial/dva-a-pul-chlapa-two-an…",4,16,2025,"""serial"""


In [529]:
df.sample(10).select(pl.col("noticka")).to_series().to_list()

['Další díl/opakování:21.12.2018 v 07:30 (Cinemax2)24.01.2019 v 07:55 (Cinemax)30.01.2019 v 03:50 (Cinemax)',
 'Další díl/opakování:02.10.2024 v 23:35 (JOJ Cinema)',
 'Další díl/opakování:17.04.2025 v 05:05 (Prima MAX)17.04.2025 v 06:55 (Prima)18.04.2025 v 05:25 (Prima MAX)18.04.2025 v 06:05 (Prima)19.04.2025 v 04:55 (Prima MAX)19.04.2025 v 06:15 (Prima)20.04.2025 v 06:45 (Prima)21.04.2025 v 05:00 (Prima MAX)',
 'Další díl/opakování:31.10.2024 v 20:15 (Film+)02.11.2024 v 18:25 (Film+)05.11.2024 v 16:50 (Film+)',
 'Další díl/opakování:18.01.2025 v 20:00 (Nova Gold)19.01.2025 v 01:40 (Nova Gold)20.01.2025 v 01:35 (Nova Gold)22.01.2025 v 04:45 (Nova Gold)25.01.2025 v 20:00 (Nova Gold)26.01.2025 v 01:25 (Nova Gold)27.01.2025 v 01:30 (Nova Gold)28.01.2025 v 04:30 (Nova Gold)',
 'Další díl/opakování:16.04.2025 v 02:50 (Prima STAR)17.04.2025 v 02:50 (Prima STAR)18.04.2025 v 02:55 (Prima STAR)19.04.2025 v 02:50 (Prima STAR)20.04.2025 v 02:50 (Prima STAR)21.04.2025 v 02:50 (Prima STAR)22.04.202

In [530]:
kanaly = df.select(pl.col("kanal")).unique().to_series().to_list()
kanaly

['Cinemax',
 'CS Film',
 'Prima',
 'Prima Show',
 'Nova Action',
 'Nova Lady',
 'Film Europe Channel',
 'ČT2',
 'Prima LOVE',
 'Cinemax2',
 'NOVA',
 'Film+',
 'ČT1',
 'Nova 2',
 'Prima STAR',
 'Prima Cool',
 'TV Barrandov',
 'Nova Cinema',
 'Nova Fun',
 'VOYO',
 'CNN Prima News',
 'ČT 3']

In [531]:
verejnopravni = [k for k in kanaly if 'ČT' in k]
print(verejnopravni)
soukrome = [k for k in kanaly if 'ČT' not in k]
print(soukrome)

['ČT2', 'ČT1', 'ČT 3']
['Cinemax', 'CS Film', 'Prima', 'Prima Show', 'Nova Action', 'Nova Lady', 'Film Europe Channel', 'Prima LOVE', 'Cinemax2', 'NOVA', 'Film+', 'Nova 2', 'Prima STAR', 'Prima Cool', 'TV Barrandov', 'Nova Cinema', 'Nova Fun', 'VOYO', 'CNN Prima News']


In [532]:
vysilane_verejnopravni = df.filter(pl.col("kanal").is_in(verejnopravni)).select(pl.col("nazev")).unique().to_series().to_list()
vysilane_soukromymi = df.filter(pl.col("kanal").is_in(soukrome)).select(pl.col("nazev")).unique().to_series().to_list()
vysilane_vsude = [x for x in vysilane_verejnopravni if x in vysilane_soukromymi]

In [533]:
aspon_na_dvou = df.group_by("nazev").agg(pl.col("kanal").unique().len()).filter(pl.col("kanal") >= 2).select(pl.col("nazev")).to_series().to_list()

In [534]:
v_poslednich_dvou_letech = df.filter(pl.col("datum").dt.year().is_in([2023,2024])).filter(pl.col("datum").dt.month().is_in([7,8])).select(pl.col("nazev")).unique().to_series().to_list()

In [535]:
x_let = df.with_columns(pl.col('datum').dt.year().alias('rok')).group_by('nazev').agg(pl.col('rok').unique().len()).filter(pl.col('rok') >= 5).select(pl.col('nazev')).unique().to_series().to_list()

In [536]:
vsude_a_x_let_pracovni = [x for x in x_let if x in aspon_na_dvou]
vsude_a_x_let = [x for x in vsude_a_x_let_pracovni if x in v_poslednich_dvou_letech]

In [537]:
zajima_nas = ['ČT1', 'ČT2', 'NOVA', 'Prima']

In [538]:
letni = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & pl.col('mesic').is_in([7,8])).group_by('nazev').len().rename({'len':'v_lete'})

In [539]:
zimni = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & (~pl.col('mesic').is_in([7,8]))).group_by('nazev').len().rename({'len':'v_zime'})

In [540]:
zimni = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & (~pl.col('mesic').is_in([6,7,8,9]))).group_by('nazev').len().rename({'len':'v_zime'})

In [541]:
kategorie = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col("nazev").is_in(vsude_a_x_let)).group_by("nazev").agg(pl.col("kategorie").mode()).explode("kategorie").drop_nulls()
kategorie = df.filter(pl.col("rok").is_between(2015,2024)).filter(pl.col("nazev").is_in(vsude_a_x_let)).group_by(['nazev','kategorie']).len().sort(by='len',descending=True).unique(subset=['nazev'], keep='first').drop('len').drop_nulls()
kategorie

nazev,kategorie
str,str
"""Legie""","""film"""
"""Čelisti""","""film"""
"""Všemocný""","""serial"""
"""Král komiků""","""film"""
"""Blbý a blbější""","""film"""
"""Láska z pasáže""","""film"""
"""Kamarád do deště 2: Příběh z B…","""film"""
"""Události za okamžik a počasí""","""film"""
"""Minority Report""","""film"""


In [542]:
pomer = letni.join(zimni, how='full', on='nazev').with_columns(
    pl.when(
        pl.col('nazev').is_null()
    ).then(
        pl.col('nazev_right')
    ).otherwise(
        pl.col('nazev')
    ).alias(
        'nazev'
    )
).with_columns(
    pl.col('v_lete').fill_null(0)
).drop(
    'nazev_right'
).with_columns(
    (pl.col('v_lete') / pl.col('v_zime')).alias('pomer')
)

In [543]:
pomer.filter(pl.col("nazev").str.contains("Lepší už"))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Lepší už to nebude""",5,null,null


In [705]:
nechceme = [
    "Šaráda",
    "Záchranáři",
    "Máma",
    "Helena",
    "Hodinový manžel",
    "Uvolněte se, prosím",
    "Vinaři",
    "Kameňák",
    "Etiketa"
]

In [707]:
filmy = kategorie.filter(pl.col('kategorie') == 'film').filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()
serialy = kategorie.filter(pl.col('kategorie') == 'serial').filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()

In [546]:
pomer.filter(pl.col('nazev').str.contains('Hříšný t'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Hříšný tanec""",4,7,0.571429


In [877]:
pomer.filter(pl.col('nazev').str.contains('Pomád'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64


In [879]:
pomer.filter(pl.col('nazev').str.contains('Vinneto'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Vinnetou - Poslední výstřel""",10,2,5.0
"""Vinnetou - Rudý gentleman""",10,2,5.0
"""Vinnetou a Old Firehand""",6,1,6.0
"""Vinnetou""",16,2,8.0
"""Vinnetou a míšenka Apanači""",12,2,6.0


In [881]:
pomer.filter(pl.col('nazev').str.contains('Shatter'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64


In [709]:
letni_filmy = pomer.filter(pl.col('nazev').is_in(filmy)).filter(pl.col('v_lete') >= 8).sort(by='pomer',descending=True).with_columns(pl.lit("film").alias("kategorie")).head(50)
letni_filmy

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Bobule""",14,null,null,"""film"""
"""Skryté zločiny osmašedesátého""",10,null,null,"""film"""
"""Za oponou""",9,null,null,"""film"""
"""Indiana Jones a dobyvatelé ztr…",13,null,null,"""film"""
"""2Bobule""",14,null,null,"""film"""
"""Indiana Jones a Poslední křížo…",13,null,null,"""film"""
"""Indiana Jones a Chrám zkázy""",13,null,null,"""film"""
"""Indiana Jones a Království kři…",13,null,null,"""film"""
"""Petrolejový princ""",12,1,12.0,"""film"""


In [711]:
letni_serialy = pomer.filter(pl.col('nazev').is_in(serialy)).filter(pl.col('v_lete') >= 25).sort(by='pomer',descending=True).with_columns(pl.lit("seriál").alias("kategorie")).head(50)
letni_serialy

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Malý pitaval z velkého města""",177,3,59.0,"""seriál"""
"""Chalupáři""",123,20,6.15,"""seriál"""
"""Dobrá Voda""",56,10,5.6,"""seriál"""
"""Pan Tau""",61,13,4.692308,"""seriál"""
"""Správná dvojka""",40,15,2.666667,"""seriál"""
"""Spongebob v kalhotách""",42,17,2.470588,"""seriál"""
"""Návštěvníci""",26,11,2.363636,"""seriál"""
"""Můj přítel Monk""",616,303,2.033003,"""seriál"""
"""Nemocnice na kraji města""",104,52,2.0,"""seriál"""


In [713]:
letni_televize = pl.concat(
    [letni_filmy, letni_serialy]
).sort(by='pomer',descending=True).filter((pl.col('pomer') >= 2.2) | (pl.col('pomer').is_null()))

In [689]:
letni_televize

nazev,v_lete,v_zime,pomer,kategorie
str,u32,u32,f64,str
"""Bobule""",14,null,null,"""film"""
"""Skryté zločiny osmašedesátého""",10,null,null,"""film"""
"""Za oponou""",9,null,null,"""film"""
"""Indiana Jones a dobyvatelé ztr…",13,null,null,"""film"""
"""2Bobule""",14,null,null,"""film"""
"""Indiana Jones a Poslední křížo…",13,null,null,"""film"""
"""Indiana Jones a Chrám zkázy""",13,null,null,"""film"""
"""Indiana Jones a Království kři…",13,null,null,"""film"""
"""Malý pitaval z velkého města""",177,3,59.0,"""seriál"""


In [551]:
pomer.filter(~pl.col('nazev').is_in(serialy) & ~pl.col('nazev').is_in(filmy)).filter(pl.col('v_lete') >= 50).sort(by='pomer',descending=True).head(20)

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Máma""",85,12,7.083333
"""Hodinový manžel""",51,15,3.4
"""Svět zvířat""",52,88,0.590909
"""Žiješ jenom 2x""",251,504,0.498016
"""V zajetí železné opony""",115,359,0.320334
"""Ta naša kapela""",96,316,0.303797
"""Zprávy v českém znakovém jazyc…",674,2466,0.273317
"""Zajímavosti z regionů""",342,1294,0.264297
"""Zprávy ve 12""",428,1652,0.25908


In [651]:
nechceme_urls = ['serial/kamenak/163648']

In [715]:
vyber_strojovy = letni_televize.filter(~pl.col('nazev').is_in(nechceme)).select(pl.col('nazev')).to_series().to_list()

graf = df.filter(
    pl.col("rok").is_between(2015,2024)
).filter(
    pl.col('nazev').is_in(vyber_strojovy)
).filter(
    ~pl.col('url').is_in(nechceme_urls)
).filter(
    pl.col("kanal").is_in(zajima_nas)
).group_by(
    ['nazev','tyden']
).len().join(
    kategorie, on="nazev", how="left"
)

graf.sort(by=['nazev','tyden'])

nazev,tyden,len,kategorie
str,i8,u32,str
"""2Bobule""",29,1,"""film"""
"""2Bobule""",30,1,"""film"""
"""2Bobule""",32,2,"""film"""
"""2Bobule""",33,10,"""film"""
"""2Bobule""",37,3,"""film"""
"""Angelika a král""",29,1,"""film"""
"""Angelika a král""",31,2,"""film"""
"""Angelika a král""",32,3,"""film"""
"""Angelika a král""",33,2,"""film"""


In [697]:
graf.filter(pl.col("nazev").str.contains("Skryté zlo"))

nazev,tyden,len,kategorie
str,i8,u32,str
"""Skryté zločiny osmašedesátého""",34,4,"""film"""
"""Skryté zločiny osmašedesátého""",35,1,"""film"""
"""Skryté zločiny osmašedesátého""",33,1,"""film"""


In [555]:
len(vyber_strojovy)

57

In [556]:
import altair as alt

In [557]:
df.sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT2""",2016-02-14 17:25:00,"""Planeta Země: Království ledu""",null,"""Dokument. Globální změny klima…",null,2,6,2016,null
"""Film+""",2021-06-29 06:00:00,"""Teleshop""",null,null,null,6,26,2021,null
"""Prima Cool""",2014-06-23 09:45:00,"""Scrubs: Doktůrci""","""04.01.2022 v 10:25 (PRO7)07.01…","""Další díl/opakování:04.01.2022…","""serial/scrubs-dokturci-scrubs/…",6,26,2014,"""serial"""
"""CS Film""",2024-03-05 23:45:00,"""Filmové minuty""","""19.01.2025 v 17:45 (CS Film)21…","""Další díl/opakování:19.01.2025…","""film/filmove-minuty/170402""",3,10,2024,"""film"""
"""Prima""",2010-12-25 20:00:00,"""Gladiátor""","""21.04.2025 v 22:50 (HBO2)""","""Další díl/opakování:21.04.2025…","""film/gladiator-gladiator/27773""",12,51,2010,"""film"""
"""Prima LOVE""",2024-09-01 15:00:00,"""Doktor z hor: Nové příběhy""","""19.01.2025 v 11:25 (Prima STAR…","""Další díl/opakování:19.01.2025…","""serial/doktor-z-hor-nove-pribe…",9,35,2024,"""serial"""
"""Nova 2""",2018-07-31 23:15:00,"""Přátelé""","""18.01.2025 v 21:40 (Nova Fun)1…","""Další díl/opakování:18.01.2025…","""serial/pratele-friends/29320""",7,31,2018,"""serial"""
"""Prima Cool""",2023-03-11 13:50:00,"""Simpsonovi XIII""","""20.01.2025 v 13:15 (Prima Cool…","""Další díl/opakování:20.01.2025…","""serial/simpsonovi-the-simpsons…",3,10,2023,"""serial"""
"""ČT1""",2011-03-18 18:45:00,"""Večerníček""","""31.01.2025 v 18:45 (ČT :D / ČT…","""Další díl/opakování:31.01.2025…","""film/vecernicek/71245""",3,11,2011,"""film"""


In [558]:
from datetime import datetime

In [717]:
roky = range(2007,2025)
prazdniny = []
for r in roky:
    prazdniny.append({'zacatek':datetime(year=r,month=7,day=1), 'konec':datetime(year=r,month=8,day=31)})

prazdniny = pl.DataFrame(prazdniny).with_columns(pl.col('zacatek').dt.week()).with_columns(pl.col('konec').dt.week())
prazdniny

df_prazdniny = []
for x in range(1,54):
    df_prazdniny.append({'tyden':x, 'prazdniny':len(prazdniny.filter((pl.col('zacatek') <= x) & (pl.col('konec') >= x)))})

df_prazdniny = pl.DataFrame(df_prazdniny).with_columns(pl.col('prazdniny') / len(roky)).filter(pl.col("prazdniny") != 0).with_columns(
    pl.col("tyden").min().alias("zacatek")
).with_columns(
    pl.col("tyden").max().alias("konec")
)
df_prazdniny

tyden,prazdniny,zacatek,konec
i64,f64,i64,i64
26,0.555556,26,36
27,1.0,26,36
28,1.0,26,36
29,1.0,26,36
30,1.0,26,36
31,1.0,26,36
32,1.0,26,36
33,1.0,26,36
34,1.0,26,36


In [560]:
prazdniny.filter((pl.col('zacatek') <= 26) & (pl.col('konec') >= 26))

zacatek,konec
i8,i8
26,35
26,35
26,35
26,35
26,35
26,35
26,35
26,35
26,35


In [561]:
df_prazdniny

tyden,prazdniny,zacatek,konec
i64,f64,i64,i64
26,0.555556,26,36
27,1.0,26,36
28,1.0,26,36
29,1.0,26,36
30,1.0,26,36
31,1.0,26,36
32,1.0,26,36
33,1.0,26,36
34,1.0,26,36


In [817]:
terminy = alt.Chart(
    df_prazdniny
).mark_rect(color='#D5B574'
).encode(
    alt.X('zacatek:Q'),
    alt.X2('konec:Q'),
    alt.Opacity('prazdniny:Q', legend=None)
)

terminy

alt.Chart(...)

In [695]:
df.filter(pl.col("nazev").str.contains("Skryté zločiny osma"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2008-08-19 21:15:00,"""Skryté zločiny osmašedesátého""",null,null,null,8,34,2008,null
"""ČT1""",2008-08-27 02:15:00,"""Skryté zločiny osmašedesátého""",null,null,null,8,35,2008,null
"""ČT2""",2009-08-18 21:20:00,"""Skryté zločiny osmašedesátého""",null,null,null,8,34,2009,null
"""ČT2""",2009-08-22 08:30:00,"""Skryté zločiny osmašedesátého""",null,null,null,8,34,2009,null
"""ČT2""",2010-08-19 17:25:00,"""Skryté zločiny osmašedesátého""",null,"""Stovky zraněných, desítky mrtv…",null,8,33,2010,null
"""ČT2""",2013-08-22 03:35:00,"""Skryté zločiny osmašedesátého""",null,"""Dokument (2008). Stovky zraněn…",null,8,34,2013,null
"""ČT2""",2014-08-19 20:55:00,"""Skryté zločiny osmašedesátého""",null,"""Dokument (2008). Opakování 27.…",null,8,34,2014,null
"""ČT2""",2014-08-28 01:10:00,"""Skryté zločiny osmašedesátého""",null,"""Dokument (2008). Stovky zraněn…",null,8,35,2014,null
"""ČT2""",2016-08-22 21:20:00,"""Skryté zločiny osmašedesátého""",null,"""Dokument (2008). Opakování 24.…",null,8,34,2016,null


In [699]:
df.filter(pl.col("nazev").str.contains("Zralé víno"))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""CS Film""",2008-04-28 18:00:00,"""Zralé víno (?)""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",4,18,2008,"""film"""
"""CS Film""",2008-05-30 22:00:00,"""Zralé víno (?)""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",5,22,2008,"""film"""
"""TV Barrandov""",2009-11-20 20:05:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",11,47,2009,"""film"""
"""TV Barrandov""",2009-11-21 09:15:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",11,47,2009,"""film"""
"""CS Film""",2010-01-22 20:00:00,"""Zralé víno (?)""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",1,3,2010,"""film"""
"""TV Barrandov""",2010-03-14 15:15:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",3,10,2010,"""film"""
"""CS Film""",2010-04-23 20:00:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",4,16,2010,"""film"""
"""CS Film""",2010-09-03 20:00:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",9,35,2010,"""film"""
"""CS Film""",2011-04-15 14:00:00,"""Zralé víno""","""20.01.2025 v 20:00 (CS Film)""","""Další díl/opakování:20.01.2025…","""film/zrale-vino/24097""",4,15,2011,"""film"""


In [745]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [887]:
obsah = alt.Chart(
    graf,
    title=alt.Title(
        'Co o prázdninách vysílá ČT1, ČT2, Nova a Prima',
        subtitle=[
            'Filmy, seriály a pořady, které v minulosti vysílaly alespoň dvě stanice, běžely v televizi',
            'v letech 2023 a 2024, a které jsou posledních deset let v červenci a srpnu vysílanější ',
            'alespoň 2,2× častěji než od října do května.'
        ]
    )
).mark_circle(
).encode(
    alt.X('tyden:Q', axis=alt.Axis(title="týden v roce", tickColor='#F1D1BB', tickOpacity=0.6, tickWidth = 0.5), scale=alt.Scale(
        domainMin = 1,
        domainMax = 52,
        padding=0,        # Remove extra padding
        nice=False        # Prevent extending to "nice" numbers
    )),
    alt.Y('nazev:N', sort=vyber_strojovy, axis=alt.Axis(title=None, tickColor='#F1D1BB', tickOpacity=0.6, tickWidth = 0.5)),
    alt.Size('len:Q', scale=alt.Scale(range=[25,300]), legend=None),
    alt.Color('kategorie:N', scale=alt.Scale(range=['#D15A41','#788A98']), legend=None)
)

finale = alt.layer(terminy, obsah).configure_view(
    stroke='transparent'
).configure_axis(grid=True, gridOpacity=0.6, gridWidth = 0.5, gridColor='#F1D1BB').properties(
    width=230,
)

finale

alt.LayerChart(...)

In [857]:
me_to_neurazi(graf=finale,kredity='vizualizace: iROZHLAS.cz | zdroj programů: fbd.cz', soubor='letni_porady')

<figure>
    <a href="https://data.irozhlas.cz/grafy/letni_porady.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy/letni_porady.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [649]:
df.filter(pl.col("nazev") == "Kameňák")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""NOVA""",2010-04-02 20:00:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",4,13,2010,"""film"""
"""NOVA""",2011-03-18 20:00:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",3,11,2011,"""film"""
"""Nova Cinema""",2011-03-19 14:00:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",3,11,2011,"""film"""
"""NOVA""",2012-03-02 20:00:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",3,9,2012,"""film"""
"""Nova Cinema""",2012-03-03 15:20:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",3,9,2012,"""film"""
"""NOVA""",2013-03-08 20:20:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",3,10,2013,"""film"""
"""Nova Cinema""",2013-03-09 16:10:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",3,10,2013,"""film"""
"""VOYO""",2013-11-23 16:55:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",11,47,2013,"""film"""
"""VOYO""",2013-11-26 12:20:00,"""Kameňák""","""12.07.2024 v 23:25 (Markíza)14…","""Další díl/opakování:12.07.2024…","""film/kamenak/30981""",11,48,2013,"""film"""


## Pořady, které zmizely

In [567]:
nebyly_v_poslednich_dvou_letech = [x for x in vsude_a_x_let_pracovni if x not in v_poslednich_dvou_letech]

In [568]:
nebyly_v_poslednich_dvou_letech

['Já to tedy beru, šéfe...!',
 'Vdáš se, a basta!',
 'Luther',
 'Nerovný zápas',
 'Doktor Flastr',
 'Kid',
 'Čertova nevěsta',
 'Anděl',
 'Drsňák Dudley',
 'Čtyřicet pušek',
 'Thelma a Louise',
 'Pes baskervillský',
 'O bílém hadovi',
 'Mezi nepřáteli',
 'Druhý muž',
 'Za onoho času',
 'Já a moje příšera',
 'Policajt v Hongkongu',
 'Nebezpečná rychlost 2: Zásah',
 'Bourák',
 'Habermannův mlýn',
 'Brontosaurus',
 'Bez svatozáře',
 'Barbora Hlavsová',
 'Krotitelé dluhů',
 'Kavárnička dříve narozených aneb Andulky jsou hezké holky',
 'Prezidentův muž',
 'Peníze těch druhých',
 'Dance World 2010',
 'Avengers: Age of Ultron',
 'Už se nebojím',
 'Zlatá hokejka',
 'Deník',
 'Jak se stal Matěj Cvrček doktorem',
 'RED 2',
 'Mumie se vrací',
 'Téměř bez šance',
 'Rozbouřená řeka',
 'Mládí v hajzlu',
 'Vosy útočí',
 'Mortal Kombat - Boj na život a na smrt',
 'Slzy slunce',
 'Cesta peklem',
 'Jak ztratit kluka v 10 dnech',
 'Pod kopyty stád',
 'Volte mě!',
 'Náhradník',
 'Nebezpečná posedlost',
 '

In [569]:
letni_kdysi = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(nebyly_v_poslednich_dvou_letech) & pl.col('mesic').is_in([7,8])).group_by('nazev').len().rename({'len':'v_lete'})
zimni_kdysi = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(nebyly_v_poslednich_dvou_letech) & (~pl.col('mesic').is_in([7,8]))).group_by('nazev').len().rename({'len':'v_zime'})

In [570]:
pomer_nevysilane = letni_kdysi.join(zimni_kdysi, how='full', on='nazev').with_columns(
    pl.when(
        pl.col('nazev').is_null()
    ).then(
        pl.col('nazev_right')
    ).otherwise(
        pl.col('nazev')
    ).alias(
        'nazev'
    )
).with_columns(
    pl.col('v_lete').fill_null(0)
).drop(
    'nazev_right'
).with_columns(
    (pl.col('v_lete') / pl.col('v_zime')).alias('pomer')
)

In [659]:
pomer_nevysilane.filter(pl.col('pomer') > 1.8).sort(by=['pomer','v_lete'],descending=True)

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Bob a Bobek na cestách""",43,3,14.333333
"""Karel Gott v Lucerně""",13,1,13.0
"""Na cestě po Jindřichohradecku""",9,1,9.0
"""Fotbal""",9,1,9.0
"""Pohádky z lesa""",8,1,8.0
"""Na cestě po Podhorácku""",8,1,8.0
"""Na cestě po Beskydech""",7,1,7.0
"""Na cestě po hanáckém Slovácku""",7,1,7.0
"""Na cestě po Vitorazsku""",7,1,7.0


In [572]:
pomer.filter(pl.col('nazev').str.contains('Hříšný ta'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Hříšný tanec""",4,7,0.571429


In [573]:
pomer.filter(pl.col('nazev').str.contains('z městečk'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64


## (dis)kontinuita

In [575]:
df.filter(pl.col('nazev').str.contains('Krotitel')).sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""Cinemax""",2017-08-20 16:20:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",8,33,2017,"""film"""
"""NOVA""",2024-03-04 01:45:00,"""Krotitelé duchů 2""","""09.04.2025 v 12:05 (HBO2)""","""Další díl/opakování:09.04.2025…","""film/krotitele-duchu-ii-ghostb…",3,10,2024,"""film"""
"""NOVA""",2022-04-30 15:00:00,"""Krotitelé duchů 2""","""09.04.2025 v 12:05 (HBO2)""","""Další díl/opakování:09.04.2025…","""film/krotitele-duchu-ii-ghostb…",4,17,2022,"""film"""
"""Film+""",2014-02-24 17:15:00,"""Krotitelé upírů""","""25.02.2014 v 13:45 (Film+)""","""Další díl/opakování:25.02.2014…","""film/krotitele-upiru-chin-gei-…",2,9,2014,"""film"""
"""ČT1""",2010-11-09 22:05:00,"""Krotitelé dluhů""","""III (9/16)""","""Po smrti manželky zjistil, že …",null,11,45,2010,null
"""NOVA""",2022-05-01 02:40:00,"""Krotitelé duchů 2""","""09.04.2025 v 12:05 (HBO2)""","""Další díl/opakování:09.04.2025…","""film/krotitele-duchu-ii-ghostb…",5,17,2022,"""film"""
"""ČT1""",2021-11-22 05:00:00,"""Krotitelé dluhů""",null,"""Hejnalovi. Život na dluh s seb…","""film/krotitele-dluhu/84924""",11,47,2021,"""film"""
"""ČT1""",2012-05-02 22:30:00,"""Krotitelé dluhů""",null,null,"""film/krotitele-dluhu/84924""",5,18,2012,"""film"""
"""Cinemax""",2018-05-16 02:25:00,"""Krotitelé duchů""","""26.01.2025 v 06:00 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ghostbust…",5,20,2018,"""film"""


In [576]:
df.select(pl.col('nazev')).sample(20)

nazev
str
"""Mambo"""
"""Konečná linky 174"""
"""Pod pokličkou"""
"""Jste to, co jíte"""
"""Zvonečkovi"""
"""Sběratelé kostí"""
"""Náš venkov"""
"""Spongebob v kalhotách"""
"""Šakal"""


In [577]:
def calculate_ratio(current_row: pl.Series, previous_row: pl.Series) -> float:
    current_items = set(current_row)
    previous_items = set(previous_row)

    if not previous_items:
        return 0.0

    common_items = current_items.intersection(previous_items)
    ratio = len(common_items) / len(previous_items)
    return ratio


In [578]:
vyvoj_ct1 = df.filter(pl.col('kanal') == 'ČT1').group_by(['rok','tyden']).agg(pl.col("nazev")).sort(by=['rok','tyden'])

In [579]:
ratios = []
for i in range(1, len(vyvoj_ct1)):
    current_row = vyvoj_ct1["nazev"][i]
    previous_row = vyvoj_ct1["nazev"][i - 1]
    ratio = calculate_ratio(current_row, previous_row)
    ratios.append(ratio)

In [580]:
ratios.insert(0, float('nan'))

In [581]:
vyvoj_ct1 = vyvoj_ct1.with_columns(pl.Series("ratio", ratios))

In [582]:
vyvoj_ct1.filter(pl.col('rok').is_between(2008,2024)).group_by('tyden').agg(pl.col('ratio').median())

tyden,ratio
i8,f64
1,0.266187
41,0.68
22,0.673913
38,0.70297
25,0.663717
19,0.650485
30,0.679144
17,0.6484375
52,0.35


In [583]:
alt.Chart(
    vyvoj_ct1.filter(pl.col('rok').is_between(2008,2024)).group_by('tyden').agg(pl.col('ratio').median()), width=800
).mark_bar(
).encode(
    alt.X('tyden:Q'),
    alt.Y('ratio:Q')
)

alt.Chart(...)

In [584]:
def calculate_ratio(current_row: pl.Series, previous_row: pl.Series) -> float:
    current_items = set(current_row)
    previous_items = set(previous_row)

    if not previous_items:
        return 0.0

    common_items = current_items.intersection(previous_items)
    ratio = len(common_items) / len(previous_items)
    return ratio
